In [3]:
from bs4 import BeautifulSoup
from IPython.display import clear_output
from itertools import chain
from datetime import date, datetime, timedelta
import random, hickle, sys, math, pickle, json,csv, re, time, os, glob
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from nltk import tokenize
from progressbar import ProgressBar
pbar = ProgressBar()
from textblob import TextBlob
import nltk
from operator import itemgetter
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from interpretable_nlp import nltk_tagger, textrank_keyword, textrank_list_keywords



def pickle_reader(filename) :
    with open(filename, 'rb') as f :
        output_df = pickle.load(f)
    return output_df

def pickle_writer(objecttosave, filename) :
    with open(filename, 'wb') as f :
        pickle.dump(objecttosave, f, protocol=4)
import pandas

NameError: name 'nltk_tagger' is not defined

## 개별 국가 dictionary

In [1]:
# Country Keyword List
def add_lowercase_country_keywords(input_keyword_list) :
    output_keyword_list = list(set(input_keyword_list + list(map(str.lower, input_keyword_list))))
    return sorted(output_keyword_list)

us_text = add_lowercase_country_keywords(['US', 'American', 'United States',  'Biden', 'Whitehouse', 'Pentagon','Blinken'])
china_text = add_lowercase_country_keywords(['China', 'Xi', 'Xi Jinping', 'Jinping', 'Chinese'])
southkorea_text = add_lowercase_country_keywords(['S.Korea' ,'South Korea','Moon Jae-in', 'Moon Jae in', 'Jae-In', 'Jaein'])
northkorea_text = add_lowercase_country_keywords(['N.Korea', 'DPRK', 'North Korea', 'Jongun','Jong-un', 'Kim Jong-Un'])
japan_text = add_lowercase_country_keywords(['Japan', 'Fumio Kishida', 'Kishida', 'Japanese'])
russia_text = add_lowercase_country_keywords(['Russia', 'Putin', 'kremlin', 'Russian'])
india_text = add_lowercase_country_keywords(['India', 'Narendra Modi', 'Modi','Republic of India', 'Bhārat', 'Bharat', 'Indian'])
uk_text = add_lowercase_country_keywords(['United Kingdom', 'Britain','UK', 'Boris Johnson', 'Westminster', 'Downing street'])
indonesia_text = add_lowercase_country_keywords(['Indonesian', 'Indonesia', 'Joko Widodo', 'Joko', 'Widodo', 'Republic of Indonesia'])
taiwan_text = add_lowercase_country_keywords(['Taiwan', 'Tsai Ing-wen', 'Tsai', 'Taipei'])
germany_text = add_lowercase_country_keywords(['Germany', 'Federal Republic of Germany', 'Berlin', 'Angela Merkel', 'Merkel' , 'Frank Walter Steinmeier', 'Steinmeier'])
mexico_text = add_lowercase_country_keywords(['Mexico', 'United Mexican States', 'Andres Manuel Lopez Obrador', 'Obrador'])
france_text = add_lowercase_country_keywords(['Frence', 'French Republic', 'Paris', 'Macron', 'Emmanuel Macron'])
australia_text = add_lowercase_country_keywords(['Australia', 'Commonwealth of Australia', 'Canberra', 'Scott John Morrison', 'Scott Morrison'])
singapore_text = add_lowercase_country_keywords(['Republic of Singapore', 'Singapore', 'Singapura', 'Lee Hsien Loong', 'Lee Hsien-Loong'])
saudi_text = add_lowercase_country_keywords(['Saudi Arabia', 'Riyadh', 'Saudi', 'Mohammed bin Salman', 'King Salman'])
rsa_text = add_lowercase_country_keywords(['Republic of South Africa', 'South Africa', 'Pretoria', 'Cyril Ramaphosa', 'Ramaphosa'])
turkey_text = add_lowercase_country_keywords(['Republic of Turkey', 'Turkey', 'Ankara', 'Recep Tayyip Erdoğan', 'Erdoğan', 'Erdogan'])
italy_text = add_lowercase_country_keywords(['Italy', 'Italian Republic', 'Rome', 'Giuseppe Conte', 'Conte', 'Sergio Mattarella'])

countrykeywords_dictionary = \
dict(zip(['USA', 'China', 'S.Korea', 'N.Korea', 'Japan', 'Russia', 'India', 'UK', 'Indonesia','Taiwan', 'Germany', 'Mexico', 'France', 'Australia', 'Singapore', 'South Africa', 'Saudi Arabia', 'Turkey', 'Italy'],
         [us_text, china_text, southkorea_text, northkorea_text, japan_text, russia_text, india_text, uk_text, indonesia_text, taiwan_text, germany_text, mexico_text, france_text, australia_text, singapore_text, rsa_text, saudi_text, turkey_text, italy_text]))

In [2]:
uk_text

['Boris Johnson',
 'Britain',
 'Downing street',
 'UK',
 'United Kingdom',
 'Westminster',
 'boris johnson',
 'britain',
 'downing street',
 'uk',
 'united kingdom',
 'westminster']

In [3]:
northkorea_text

['DPRK',
 'Jong-un',
 'Jongun',
 'Kim Jong-Un',
 'N.Korea',
 'North Korea',
 'dprk',
 'jong-un',
 'jongun',
 'kim jong-un',
 'n.korea',
 'north korea']

In [4]:
china_text

['China',
 'Chinese',
 'Jinping',
 'Xi',
 'Xi Jinping',
 'china',
 'chinese',
 'jinping',
 'xi',
 'xi jinping']

In [5]:
us_text

['American',
 'Biden',
 'Blinken',
 'Pentagon',
 'US',
 'United States',
 'Whitehouse',
 'american',
 'biden',
 'blinken',
 'pentagon',
 'united states',
 'us',
 'whitehouse']

In [6]:
japan_text

['Fumio Kishida',
 'Japan',
 'Japanese',
 'Kishida',
 'fumio kishida',
 'japan',
 'japanese',
 'kishida']

In [7]:
italy_text

['Conte',
 'Giuseppe Conte',
 'Italian Republic',
 'Italy',
 'Rome',
 'Sergio Mattarella',
 'conte',
 'giuseppe conte',
 'italian republic',
 'italy',
 'rome',
 'sergio mattarella']

In [15]:
japan_text

['Fumio Kishida',
 'Japan',
 'Japanese',
 'Kishida',
 'fumio kishida',
 'japan',
 'japanese',
 'kishida']

In [36]:

us_text.pop(-2)
china_text.pop(-2)
northkorea_text.pop(-7)
uk_text.pop(-3)
japan_text.pop(-5)

NameError: name 'italy_text' is not defined

## Ner 모델을 돌리기 위한 함수

In [11]:
def remain_onesentences(input_article) :
    input_article = ' '.join(input_article.split())
    first_sentence = tokenize.sent_tokenize(input_article)[0]
    return first_sentence

def splitby_threesentences(input_article) :
    input_article = ' '.join(input_article.split())
    spit_article = tokenize.sent_tokenize(input_article)
    iterated_split_article = iter(spit_article)
    joined3_article = [i + next(iterated_split_article, '') + next(iterated_split_article, '') for i in iterated_split_article]
    return joined3_article

def morethan_two_countries(input_text) :
    list_of_countries = []
    counter = 0
    for each_country_keywords in countrykeywords_dictionary.keys() :
        if len(re.findall('|'.join(countrykeywords_dictionary[each_country_keywords]), input_text))>0 :
            counter += 1
            list_of_countries.append(each_country_keywords)
    list_of_countries = ' / '.join(list(set(list_of_countries)))
            
    if counter >= 3 :
        return False, list_of_countries
    elif counter > 1 :
        return True, list_of_countries
    else :
        return False, list_of_countries

vectorized_morethan_two_countries = np.vectorize(morethan_two_countries, otypes=[list])

def sentence_to_nerlist(input_sentence) :
    try :
        ner_output_list = [each for each in model.predict(input_sentence) if each['tag']!='O']

        if len(ner_output_list)!=0 :
            ner_output_df = pd.DataFrame(ner_output_list)
            ner_output_df['first_tag'] = list(map(itemgetter(0), ner_output_df['tag'].str.split('-')))
            ner_output_df['second_tag'] = list(map(itemgetter(1), ner_output_df['tag'].str.split('-')))
            entity_list = []
            for idx in range(len(ner_output_df)) :
                temp_entity_list = []
                if idx == len(ner_output_df) :
                    temp_entity_list.append(ner_output_df['word'][idx])
                elif ner_output_df['first_tag'][idx]=='I' :
                    pass
                else :
                    temp_idx = idx
                    temp_entity_list.append(ner_output_df['word'][temp_idx])
                    try :
                        while ner_output_df['first_tag'][temp_idx+1]=='I' :
                            temp_idx += 1
                            temp_entity_list.append(ner_output_df['word'][temp_idx])
                    except KeyError :
                        pass
                if len(temp_entity_list)!=0 :
                    entity_list.append(' '.join(temp_entity_list))
        else :
            entity_list = []
    except :
        entity_list = []
    return entity_list

def corpus_to_nerlist(input_corpus) :
    return list(set(chain.from_iterable(map(sentence_to_nerlist, input_corpus))))

In [12]:
def sort_sentence_importance(input_text, standard='mean', topn=3, countryfilter=False) :
    textrank_dict = dict(textrank_keyword(input_text, topk=30))
    output_list = []
    for idx, each_sentence in enumerate(sent_tokenize(input_text)) :
        sentence_value = []
        pattern = re.compile('photo|Photo|Related article|RELATED ARTICLES|Xinhua')        
        if not pattern.search(each_sentence):
            for each_token in word_tokenize(each_sentence) :
                try :
                    sentence_value.append(textrank_dict[each_token])
                except :
                    sentence_value.append(0)
                    pass
                binary_sentence_value = list(np.vectorize(lambda x : 1 if x>0 else 0)(sentence_value))

            importance_mean = np.mean(sentence_value)
            importance_sum = np.sum(sentence_value)
            importance_ratio = np.mean(binary_sentence_value)
            output_list.append((idx, each_sentence, importance_mean, importance_sum, importance_ratio))
        
    output_df = pd.DataFrame(output_list, columns=['idx', 'sentence', 'mean', 'sum', 'ratio'])
    if countryfilter :
        output_df['numcountryfilter'] = np.vectorize(lambda x : morethan_two_countries(x)[0])(output_df['sentence'])
        output_df = output_df[output_df['numcountryfilter']==True]
    output_df = output_df.sort_values(standard, ascending=False).reset_index(drop=True)
    return output_df['sentence'][:topn].tolist()

### 주의

- 일단은 전체 기사의 sentence에 ner을 적용하는 quasiNER_extractor1을 남겨두었습니다.
- 만일 사용자가 제목과 첫번째 문장에만 ner을 적용하고 싶다면, quasiNER_extractor를 사용하세요.

In [13]:
def quasiNER_extractor1(dataframe, nameof_articlebody_column) :
    dataframe['lowercase_' + nameof_articlebody_column] = dataframe[nameof_articlebody_column]
    quasinerdf_output_list = []
    for label in range(len(dataframe)):
        doc_id = label
        try :           
            split3_article = splitby_threesentences(dataframe['lowercase_' + nameof_articlebody_column][label])
            output_list = vectorized_morethan_two_countries(split3_article)
            output_df = pd.DataFrame.from_records(output_list, columns=['isvalid', 'list_of_countries'])
            output_df['sentence'] = split3_article
            output_df['doc_id'] = doc_id
            quasinerdf_output_list.append(output_df)
        except :
            pass
    quasinerdf_output_df = pd.concat(quasinerdf_output_list, axis=0)
    quasinerdf_output_df = quasinerdf_output_df[quasinerdf_output_df['isvalid']==True].reset_index(drop=True)
    del quasinerdf_output_df['isvalid']
    return quasinerdf_output_df

In [14]:
def quasiNER_extractor(dataframe, nameof_articlebody_column) :
    dataframe['lowercase_' + nameof_articlebody_column] = dataframe[nameof_articlebody_column]
    quasinerdf_output_list = []
    for label in range(len(dataframe['lowercase_' + nameof_articlebody_column])):
        doc_id = label
        try:
            fisrt_sentence = remain_onesentences(dataframe['lowercase_' + nameof_articlebody_column][label])
            add_sentence = dataframe['title'][label] + '.'+ fisrt_sentence
            output_list = vectorized_morethan_two_countries(add_sentence)
            output_df = pd.DataFrame.from_records([output_list], columns=['isvalid', 'list_of_countries'])
            output_df['sentence'] = add_sentence
            output_df['doc_id']= doc_id
            quasinerdf_output_list.append(output_df)
        except:
            continue
    quasinerdf_output_df = pd.concat(quasinerdf_output_list, axis=0)
    quasinerdf_output_df = quasinerdf_output_df[quasinerdf_output_df['isvalid']==True].reset_index(drop=True)
    del quasinerdf_output_df['isvalid']
    return quasinerdf_output_df

In [15]:
def quasiNER_extractor3(dataframe, nameof_articlebody_column) :
    dataframe['lowercase_' + nameof_articlebody_column] = dataframe[nameof_articlebody_column]
    quasinerdf_output_list = []
    for label in range(len(dataframe['lowercase_' + nameof_articlebody_column])):
        doc_id = label
        try:
            input_text = dataframe['lowercase_' + nameof_articlebody_column][label]
            sentence = ' '.join(sort_sentence_importance(input_text, standard='mean', topn=3))
            output_list = vectorized_morethan_two_countries(sentence)
            output_df = pd.DataFrame.from_records([output_list], columns=['isvalid', 'list_of_countries'])
            output_df['input_text'] = sentence
            output_df['doc_id']= doc_id
            quasinerdf_output_list.append(output_df)
        except:
            continue
    quasinerdf_output_df = pd.concat(quasinerdf_output_list, axis=0)
    quasinerdf_output_df = quasinerdf_output_df[quasinerdf_output_df['isvalid']==True].reset_index(drop=True)
    del quasinerdf_output_df['isvalid']
    return quasinerdf_output_df

## Loading news_data

- 여기에서 ner를 통해 처리해야할 뉴스 기사를 넣으시면 됩니다.
- 예시에서는 카테고리 전처리도 포함하고 있습니다.

### fox news

In [20]:
fox = pd.read_excel('C:/Users/13a71/Dropbox/BigDataDiplomacy/Data/2021/raw_data/allnews_df.xlsx')

In [18]:
fox = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/us/fox_complete.xlsx')
fox['title'] = fox['title'].str.replace('\n', ' ')
fox['content'] = fox['content'].str.replace('\n', ' ')
fox['source'] = "FoxNews"
## remove nan category. since category selection
fox = fox.dropna(axis=0).reset_index(drop=True)
Counter(fox['category']).most_common(100)
fox = fox[fox['category'].str.contains("trump|iran|whitehouse|corona|kingdom|europe|africa|security|military|china|hongkong|asia|france|russia")].reset_index(drop=True)
fox['doc_id'] = range(0,len(fox))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/SNUda/Downloads/juwon/raw_data/us/fox_complete.xlsx'

In [16]:
fox = fox.loc[fox["date"] >= "2020-08-08"].reset_index(drop=True)
fox = fox.loc[fox["date"] <= "2020-08-24"].reset_index(drop=True)


In [26]:
ner_df = quasiNER_extractor3(fox, 'content')

In [27]:
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        ner_df['sententce_ner'][a] = [each for each in model.predict(ner_df['input_text'][a]) if each['tag']!='O']
    except :
        ner_df['sententce_ner'][a] = ""

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 512). Running this sequence through the model will result in indexing errors
C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
#nyt = pd.merge(ner_df, fox)
fox.to_excel('C:/Users/13a71/Dropbox/noun_extraction.xlsx', index = False)

### wahington post

In [28]:
wp = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/us/wp_complete.xlsx')

In [29]:
wp

,date,title,category,author,content,url,subtitle,source,doc_id
0,2020-05-15,The coronavirus is upending cash economies. Mo...,Africa,Miriam Berger,When Lagos went into lockdown last month banks...,https://wapo.st/2TdcczP,NaN,WashingtonPost,0
1,2020-05-16,Postcards from a nervous world: Tales from the...,World,Brian Murphy,And what comes next? A reopening in some form ...,https://wapo.st/2X0Xkpw,NaN,WashingtonPost,1
2,2020-05-16,Alaska lawmaker says Hitler was not white supr...,Politics,Hannah Knowles,The uproar began when an Alaska lawmaker email...,https://wapo.st/3dYhQhr,NaN,WashingtonPost,2
3,2020-05-16,"On the way to the store, a mask giveaway to pr...",National,NaN,MINNEAPOLIS — The volunteers gather every Satu...,https://wapo.st/2X5b7uX,NaN,WashingtonPost,3
4,2020-05-16,"If this pandemic is a war, Burr is the war pro...",Opinions,Helaine Olen,President Trump has more than once compared th...,https://wapo.st/2Z7GOGH,NaN,WashingtonPost,4
...,...,...,...,...,...,...,...,...,...
13075,2020-07-15,"‘Let’s stop this nonsense,’ Fauci says of fede...",National,Kim Bellware,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,Tulsa poised to mandate mask-wearing after rec...,WashingtonPost,14119
13076,2020-07-15,Coronavirus could be ‘under control’ in weeks ...,National,Miriam Berger,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,"With babies due, two Washington Capitals prepa...",WashingtonPost,14152
13077,2020-07-13,"California, Oregon roll back reopenings as new...",National,Siobhán O'Grady,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,The NBA has arrived at Disney. Here’s what it ...,WashingtonPost,14199
13078,2020-07-09,"As coronavirus hospitalizations climb, Trump s...",National,Kim Bellware,The Washington Post is providing this importa...,https://wapo.st/38EVf7S,"NFL preseason game protocols: no jersey swaps,...",WashingtonPost,14362


In [30]:
#wp['date'] = wp['date'].dt.strftime('%Y-%m-%d')
wp = wp.loc[wp["date"] >= "2019-07-01"].reset_index(drop=True)
wp = wp.loc[wp["date"] <= "2020-07-31"].reset_index(drop=True)   
wp['title'] = wp['title'].str.replace('\n', ' ')
wp['content'] = wp['content'].str.replace('\n', ' ')
wp['source'] = "WashingtonPost"
wp = wp[pd.notnull(wp['category'])]
wp = wp[wp['category'].str.contains("Politics|National|Security|Europe|Health|Middle East|World|Asia|Opinion|opinion|Africa|Military")].reset_index(drop=True)
wp['doc_id'] = range(0,len(wp))
wp = wp.drop_duplicates(subset=['title'])
wp =wp.dropna(subset=['title']).reset_index(drop=True) 

In [31]:
ner_df = quasiNER_extractor3(wp, 'content')
ner_df['sententce_ner'] = ''

In [32]:
model.predict(ner_df['input_text'][0])

[{'word': 'Yet', 'tag': 'O', 'confidence': 0.9998997449874878},
 {'word': 'the', 'tag': 'O', 'confidence': 0.9999589920043945},
 {'word': 'U.S.', 'tag': 'B-LOC', 'confidence': 0.9997699856758118},
 {'word': 'death', 'tag': 'O', 'confidence': 0.999971866607666},
 {'word': 'toll', 'tag': 'O', 'confidence': 0.9999744892120361},
 {'word': 'from', 'tag': 'O', 'confidence': 0.9999696016311646},
 {'word': 'covid-19', 'tag': 'O', 'confidence': 0.9992207288742065},
 {'word': 'exceeds', 'tag': 'O', 'confidence': 0.9999674558639526},
 {'word': '85000', 'tag': 'O', 'confidence': 0.9999454021453857},
 {'word': 'while', 'tag': 'O', 'confidence': 0.9999721050262451},
 {'word': 'South', 'tag': 'B-LOC', 'confidence': 0.9997738003730774},
 {'word': 'Korea', 'tag': 'I-LOC', 'confidence': 0.9992165565490723},
 {'word': '’', 'tag': 'O', 'confidence': 0.9999679327011108},
 {'word': 's', 'tag': 'O', 'confidence': 0.9999719858169556},
 {'word': 'fatalities', 'tag': 'O', 'confidence': 0.9999758005142212},
 {'w

In [33]:
wp

,date,title,category,author,content,url,subtitle,source,doc_id,lowercase_content
0,2020-05-15,The coronavirus is upending cash economies. Mo...,Africa,Miriam Berger,When Lagos went into lockdown last month banks...,https://wapo.st/2TdcczP,NaN,WashingtonPost,0,When Lagos went into lockdown last month banks...
1,2020-05-16,Postcards from a nervous world: Tales from the...,World,Brian Murphy,And what comes next? A reopening in some form ...,https://wapo.st/2X0Xkpw,NaN,WashingtonPost,1,And what comes next? A reopening in some form ...
2,2020-05-16,Alaska lawmaker says Hitler was not white supr...,Politics,Hannah Knowles,The uproar began when an Alaska lawmaker email...,https://wapo.st/3dYhQhr,NaN,WashingtonPost,2,The uproar began when an Alaska lawmaker email...
3,2020-05-16,"On the way to the store, a mask giveaway to pr...",National,NaN,MINNEAPOLIS — The volunteers gather every Satu...,https://wapo.st/2X5b7uX,NaN,WashingtonPost,3,MINNEAPOLIS — The volunteers gather every Satu...
4,2020-05-16,"If this pandemic is a war, Burr is the war pro...",Opinions,Helaine Olen,President Trump has more than once compared th...,https://wapo.st/2Z7GOGH,NaN,WashingtonPost,4,President Trump has more than once compared th...
...,...,...,...,...,...,...,...,...,...,...
13075,2020-07-15,"‘Let’s stop this nonsense,’ Fauci says of fede...",National,Kim Bellware,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,Tulsa poised to mandate mask-wearing after rec...,WashingtonPost,13075,The Washington Post is providing this importa...
13076,2020-07-15,Coronavirus could be ‘under control’ in weeks ...,National,Miriam Berger,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,"With babies due, two Washington Capitals prepa...",WashingtonPost,13076,The Washington Post is providing this importa...
13077,2020-07-13,"California, Oregon roll back reopenings as new...",National,Siobhán O'Grady,The Washington Post is providing this importa...,https://www.washingtonpost.com/nation/2020/07/...,The NBA has arrived at Disney. Here’s what it ...,WashingtonPost,13077,The Washington Post is providing this importa...
13078,2020-07-09,"As coronavirus hospitalizations climb, Trump s...",National,Kim Bellware,The Washington Post is providing this importa...,https://wapo.st/38EVf7S,"NFL preseason game protocols: no jersey swaps,...",WashingtonPost,13078,The Washington Post is providing this importa...


In [34]:

for a in range(len(ner_df)):
    try :
        ner_df['sententce_ner'][a] = [each for each in model.predict(ner_df['input_text'][a]) if each['tag']!='O']
    except :
        ner_df['sententce_ner'][a] = ""
wp = pd.merge(ner_df, wp)
wp.to_excel('wp_ner.xlsx', index = False)
    

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
Token indices sequence length is longer than the specified maximum sequence length for this model (2338 > 512). Running this sequence through the model will result in indexing errors
C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
Token indices sequence length is longer than the specified maximum sequence length for this model (1310 > 512). Running this sequen

### CNN

In [109]:
cnn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/us/cnn_complete.xlsx')

In [110]:
cnn['date'] = pd.to_datetime(cnn['date'], errors='coerce')
cnn['date'] = cnn['date'].dt.strftime('%Y-%m-%d')
cnn = cnn.loc[cnn["date"] >= "2019-07-01"].reset_index(drop=True)
cnn = cnn.loc[cnn["date"] <= "2020-07-31"].reset_index(drop=True) 
sentence_df = pd.DataFrame(columns=['doc_id', 'sentence', 'sententce_ner'])
cnn['title'] = cnn['title'].str.replace('\n', ' ')
cnn['content'] = cnn['content'].str.replace('\n', ' ')
cnn['source'] = "CNN"
cnn = cnn.dropna(axis=0).reset_index(drop=True)
Counter(cnn['category']).most_common(30)
cnn = cnn[cnn['category'].str.contains("politics|us|opinions|health|world|asia|europe|uk|middleeast|africa|australia|india|china")].reset_index(drop=True)
cnn['doc_id'] = range(0,len(cnn))
cnn = cnn.drop_duplicates(subset=['title'])
cnn =cnn.dropna(subset=['title']).reset_index(drop=True) 
cnn['content'] = cnn['content'].str.replace(r'“|”|’', '\'')

In [111]:
Counter(cnn['category']).most_common(30)

[('politics', 9256),
 ('us', 7279),
 ('health', 1459),
 ('world', 1162),
 ('business', 1133),
 ('opinions', 1093),
 ('asia', 860),
 ('europe', 515),
 ('uk', 305),
 ('africa', 197),
 ('middleeast', 195),
 ('australia', 131),
 ('india', 25),
 ('china', 18),
 ('_politics-zone-injection', 1),
 ('app-politics-section', 1)]

In [112]:
ner_df = quasiNER_extractor3(cnn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
cnn = pd.merge(ner_df, cnn)
cnn.to_excel('cnn_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### NewYorkTimes

In [113]:
nyt = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/us/nyt_complete.xlsx')

In [114]:
nyt['date'] = pd.to_datetime(nyt['date'], errors='coerce')
nyt['date'] = nyt['date'].dt.strftime('%Y-%m-%d')
nyt = nyt.loc[nyt["date"] >= "2019-07-01"].reset_index(drop=True)
nyt = nyt.loc[nyt["date"] <= "2020-07-31"].reset_index(drop=True)
sentence_df = pd.DataFrame(columns=['doc_id', 'sentence', 'sententce_ner'])
nyt['title'] = nyt['title'].str.replace('\n', ' ')
nyt['content'] = nyt['content'].str.replace('\n', ' ')
nyt['source'] = "NewyorkTimes"
nyt = nyt.dropna(axis=0).reset_index(drop=True)
nyt = nyt[nyt['category'].str.contains("briefing|us|opinion|health|world")].reset_index(drop=True)
nyt = nyt[~nyt['category'].str.contains("business")].reset_index(drop=True)
nyt = nyt.drop_duplicates(subset=['title'])
nyt =nyt.dropna(subset=['title']).reset_index(drop=True) 

nyt = nyt.loc[nyt["date"] >= "2019-07-01"].reset_index(drop=True)
nyt = nyt.loc[nyt["date"] <= "2020-07-31"].reset_index(drop=True)
nyt['doc_id'] = range(0,len(nyt))

In [115]:
ner_df = quasiNER_extractor3(nyt, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
nyt = pd.merge(ner_df, nyt)
nyt.to_excel('nyt_ner_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


### 환구시보

In [26]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/chn.xlsx')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn = chn.dropna(axis=0).reset_index(drop=True)

In [27]:
#chn = chn[~chn['content'].str.contains("File photo")].reset_index(drop=True)

chn['doc_id'] = range(0,len(chn))

In [28]:
chn

,url,title,date,content,category,source,scraping_date,doc_id
0,https://www.globaltimes.cn/content/1156409.shtml,Anonymity plea,2019-07-01,British singer Cliff Richard gestures as he le...,EUROPE,Global Times(China),2020-07-09 19:03:02,0
1,https://www.globaltimes.cn/content/1156411.shtml,China praises historic Trump-Kim meeting; N.K...,2019-07-01,North Korea's leader Kim Jong-un speaks as he ...,"ASIA-PACIFIC,WORLDFOCUS",Global Times(China),2020-07-09 19:03:10,1
2,https://www.globaltimes.cn/content/1156433.shtml,"China welcomes DPRK, US, South Korea leaders'...",2019-07-01,China on Monday said it welcomed the recent tr...,DIPLOMACY,Global Times(China),2020-07-09 19:04:42,2
3,https://www.globaltimes.cn/content/1156429.shtml,Chinese premier meets Bulgarian president,2019-07-01,Chinese Premier Li Keqiang on Monday met with ...,DIPLOMACY,Global Times(China),2020-07-09 19:04:25,3
4,https://www.globaltimes.cn/content/1156428.shtml,Chinese premier meets Georgian PM,2019-07-01,Premier Li Keqiang on Monday met with Georgian...,DIPLOMACY,Global Times(China),2020-07-09 19:04:21,4
...,...,...,...,...,...,...,...,...
7541,https://www.globaltimes.cn/content/1196252.shtml,In pics: closed market during lockdown in Lah...,2020-07-31,A closed market is seen during lockdown in eas...,WORLD,Global Times(China),2020-08-16 04:36:31,7541
7542,https://www.globaltimes.cn/content/1196255.shtml,Lights lit up for upcoming celebrations for 5...,2020-07-31,Red and white lights light up the facade of th...,WORLD,Global Times(China),2020-08-16 04:36:37,7542
7543,https://www.globaltimes.cn/content/1196277.shtml,"Mongolia to hand over first batch of 30,000 s...",2020-07-31,File photo:Xinhua Mongolia is planning to give...,"DIPLOMACY,CHINAFOCUS",Global Times(China),2020-08-16 04:37:20,7543
7544,https://www.globaltimes.cn/content/1196253.shtml,"People cool down on Danube River in Belgrade,...",2020-07-31,People cool down on the Danube River at the po...,WORLD,Global Times(China),2020-08-16 04:36:33,7544


In [29]:
Counter(chn['category']).most_common(30)

[('WORLD', 2192),
 ('DIPLOMACY', 828),
 ('EUROPE', 761),
 ('ASIA-PACIFIC', 723),
 ('HK/MACAO/TAIWAN', 311),
 ('AMERICAS,WORLDFOCUS', 192),
 ('AFRICA', 184),
 ('MILITARY', 180),
 ('EUROPE,WORLDFOCUS', 159),
 ('ASIA-PACIFIC,WORLDFOCUS', 139),
 ('DIPLOMACY,CHINAFOCUS', 105),
 ('CROSS-BORDERS,WORLDFOCUS', 94),
 ('ODDNEWS,WORLD', 80),
 ('POLITICS,DIPLOMACY', 58),
 ('MID-EAST,WORLDFOCUS', 56),
 ('AMERICAS,EYEONWORLD', 48),
 ('EUROPE,EYEONWORLD', 44),
 ('ASIA-PACIFIC,EYEONWORLD', 44),
 ('MILITARY,CHINAFOCUS', 42),
 ('CROSS-BORDERS,EYEONWORLD', 42),
 ('POLITICS,HK/MACAO/TAIWAN', 41),
 ('AMERICAS,WORLDFOCUS,EYEONWORLD', 38),
 ('EUROPE,AMERICAS', 36),
 ('CENTRAL&SOUTHASIA,WORLDFOCUS', 36),
 ('EYEONWORLD', 31),
 ('EUROPE,WORLDFOCUS,EYEONWORLD', 31),
 ('AFRICA,WORLDFOCUS', 30),
 ('DIPLOMACY,MILITARY', 30),
 ('HK/MACAO/TAIWAN,LATESTNEWS', 24),
 ('MID-EAST,EYEONWORLD', 24)]

In [30]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('chn_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [31]:
chn

,list_of_countries,input_text,doc_id,sententce_ner,url,title,date,content,category,source,scraping_date,lowercase_content
0,USA / N.Korea,"Before the Sunday meeting, Kim and Trump held ...",7,"[{'word': 'Kim', 'tag': 'B-PER', 'confidence':...",https://www.globaltimes.cn/content/1156329.shtml,DPRK's state media says breakthrough achieved...,2019-07-01,"The historic meeting between Kim Jong Un, top ...",ASIA-PACIFIC,Global Times(China),2020-07-09 18:57:22,"The historic meeting between Kim Jong Un, top ..."
1,Taiwan / China,People attend a hot air balloon festival in Ta...,11,"[{'word': 'Taitung', 'tag': 'B-LOC', 'confiden...",https://www.globaltimes.cn/content/1156342.shtml,Hot air balloon festival held in China's Taiwan,2019-07-01,People attend a hot air balloon festival in Ta...,WORLD,Global Times(China),2020-07-09 18:58:25,People attend a hot air balloon festival in Ta...
2,China / UK,"Through lectures, teaching workshops and discu...",13,"[{'word': 'Chinese', 'tag': 'B-MISC', 'confide...",https://www.globaltimes.cn/content/1156400.shtml,Local Chinese language teachers trained in Lo...,2019-07-01,Photo: Sun Wei/GT The second special training ...,EUROPE,Global Times(China),2020-07-09 19:02:26,Photo: Sun Wei/GT The second special training ...
3,S.Korea / Japan,"South Korean Minister of Trade, Industry and E...",20,"[{'word': 'South', 'tag': 'B-MISC', 'confidenc...",https://www.globaltimes.cn/content/1156369.shtml,S. Korea to file suit with WTO against Japan'...,2019-07-01,"South Korean Minister of Trade, Industry and E...",ASIA-PACIFIC,Global Times(China),2020-07-09 19:00:21,"South Korean Minister of Trade, Industry and E..."
4,Turkey / China,"Turkey and China, the world's most ancient civ...",23,"[{'word': 'Turkey', 'tag': 'B-LOC', 'confidenc...",https://www.globaltimes.cn/content/1156357.shtml,"Turkey, China share a vision for future",2019-07-01,Photo: VCGTurkey and China have major responsi...,"VIEWPOINT,WORLDFOCUS",Global Times(China),2020-07-09 18:59:30,Photo: VCGTurkey and China have major responsi...
...,...,...,...,...,...,...,...,...,...,...,...,...
1238,India / China,Supporters of India's Congress party display p...,7512,"[{'word': 'India', 'tag': 'B-LOC', 'confidence...",https://www.globaltimes.cn/content/1196242.shtml,Indiaâs Rafale fighter jets have no chance ...,2020-07-30,Supporters of India's Congress party display p...,MILITARY,Global Times(China),2020-08-16 04:36:13,Supporters of India's Congress party display p...
1239,USA / China,The H-6J's weapon capacity is about twice that...,7520,"[{'word': 'H-6J', 'tag': 'B-MISC', 'confidence...",https://www.globaltimes.cn/content/1196230.shtml,PLA Navyâs new bomber debuts in South China...,2020-07-30,An H-6 bomber attached to an aviation regiment...,MILITARY,Global Times(China),2020-08-16 04:35:48,An H-6 bomber attached to an aviation regiment...
1240,USA / China,Ren said that China has indisputable sovereign...,7529,"[{'word': 'Ren', 'tag': 'B-PER', 'confidence':...",https://www.globaltimes.cn/content/1196185.shtml,US military provocations in S.China Sea to st...,2020-07-30,File photo shows a Chinese H-6K bomber patroll...,"POLITICS,MILITARY",Global Times(China),2020-08-16 04:34:29,File photo shows a Chinese H-6K bomber patroll...
1241,China / UK,There has been blatant interference from Brita...,7534,"[{'word': 'Britain', 'tag': 'B-LOC', 'confiden...",https://www.globaltimes.cn/content/1196270.shtml,"Britain's Xinjiang move ""seriously poisoned"" ...",2020-07-31,National flags of China and Britain are seen i...,CHINA-EUROPE,Global Times(China),2020-08-16 04:37:07,National flags of China and Britain are seen i...


### 가디언

In [122]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/uk.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn = chn.dropna(axis=0).reset_index(drop=True)
chn['doc_id'] = range(0,len(chn))

In [123]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('UK_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
Token indices sequence length is longer than the specified maximum sequence length for this model (830 > 512). Running this sequence through the model will result in indexing errors
C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [124]:
chn

,list_of_countries,input_text,doc_id,sententce_ner,category,content,date,scrap_time,source,subtitle,title,url,lowercase_content
0,Australia / USA,Scott Morrison praises Donald Trump's politica...,4,"[{'word': 'Scott', 'tag': 'B-PER', 'confidence...",World,Donald Trump called Scott Morrison to ask for ...,2019-10-01,2020-07-07-27 20:39:13,Guardian,President urged Scott Morrison to help gather ...,Australian government agreed to help Donald Tr...,https://www.theguardian.com/us-news/2019/oct/0...,Donald Trump called Scott Morrison to ask for ...
1,China / UK,The Guardian view on the People’s Republic of ...,10,"[{'word': 'The', 'tag': 'B-ORG', 'confidence':...",World,China has celebrated 70 years of Communist par...,2019-10-01,2020-07-07-27 20:38:56,Guardian,New weapons make public debut in sweeping mili...,China celebrates 70th anniversary as Xi warns ...,https://www.theguardian.com/world/2019/oct/01/...,China has celebrated 70 years of Communist par...
2,China / UK,Hong Kong has been gripped by protests since J...,19,"[{'word': 'Hong', 'tag': 'B-LOC', 'confidence'...",World,Hong Kong police have shot a protester with li...,2019-10-01,2020-07-07-27 20:38:43,Guardian,Violence erupts as thousands protest on 70th a...,Hong Kong protester shot with live round durin...,https://www.theguardian.com/world/2019/oct/01/...,Hong Kong police have shot a protester with li...
3,China / UK,Control of Hong Kong is handed back to China f...,20,"[{'word': 'Hong', 'tag': 'B-LOC', 'confidence'...",World,October 1 was meant to be a carefully choreogr...,2019-10-01,2020-07-07-27 20:38:34,Guardian,Beijing’s carefully planned celebrations turne...,Hong Kong protesters rain on China’s anniversa...,https://www.theguardian.com/world/2019/oct/01/...,October 1 was meant to be a carefully choreogr...
4,UK / USA,The Home Office says:“We welcome international...,28,"[{'word': 'Home', 'tag': 'B-ORG', 'confidence'...",World,"Amber Murrey, an American academic, was “ecsta...",2019-10-01,2020-07-09-27 03:28:02,Guardian,US family split as Home Office makes it increa...,Oxford professor’s children refused visas to j...,https://www.theguardian.com/education/2019/oct...,"Amber Murrey, an American academic, was “ecsta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,UK / USA,Donald Trump’s administration has blocked the ...,15878,,World,"Liam Fox, Britain’s former international trade...",2020-07-08,2020-08-17-33 06:09:29,Guardian,Government describes former international trad...,UK nominates Liam Fox to lead World Trade Orga...,https://www.theguardian.com/world/2020/jul/08/...,"Liam Fox, Britain’s former international trade..."
1662,Mexico / USA,“We didn’t fight. “He didn’t score any goals a...,15884,,World,It was a high-stakes encounter for a man takin...,2020-07-09,2020-08-17-33 06:08:38,Guardian,The Mexican president faced criticism before h...,Amlo unscathed after Trump meeting but snags c...,https://www.theguardian.com/world/2020/jul/09/...,It was a high-stakes encounter for a man takin...
1663,UK / USA,"Brazil has 1.7 million cases and 67,964 deaths...",15892,,World,The number of confirmed coronavirus infections...,2020-07-09,2020-08-17-33 06:09:10,Guardian,Infections globally rising by a million a week...,Coronavirus report: global cases pass 12m as U...,https://www.theguardian.com/world/2020/jul/09/...,The number of confirmed coronavirus infections...
1664,France / UK,International aviation and shipping likely to ...,15902,,World,There is a one-in-five chance annual global te...,2020-07-09,2020-08-17-33 06:09:16,Guardian,Experts say new climate data shows how close w...,Global temperatures likely to hit at least 1C ...,https://www.theguardian.com/environment/2020/j...,There is a one-in-five chance annual global te...


### speigel

In [19]:

chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/ger.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn['doc_id'] = range(0,len(chn))

In [29]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('ger_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### hindustan

In [30]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/ind.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn = chn.dropna(axis=0).reset_index(drop=True)
chn['doc_id'] = range(0,len(chn))

In [31]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('ind_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [32]:
chn

,list_of_countries,input_text,doc_id,sententce_ner,category,content,date,scrap_time,source,subtitle,title,url,lowercase_content
0,China / USA,"This is taking.........place in TRADE, it’s ta...",2,"[{'word': 'Military', 'tag': 'B-MISC', 'confid...",World,US President Donald Trump on Tuesday congratul...,2019-10-01,2020-08-16-32 12:08:37,Hindustan Times,“Congratulations to President Xi and the Chine...,Donald Trump congratulates China on National D...,https://www.hindustantimes.com/world-news/dona...,US President Donald Trump on Tuesday congratul...
1,Taiwan / China,"Besides the DF-41, China also unveiled a steal...",4,"[{'word': 'DF-41', 'tag': 'B-MISC', 'confidenc...",World,China on Tuesday brought out its big guns and ...,2019-10-01,2020-08-16-32 12:08:40,Hindustan Times,"The military parade of China comprising 15,000...","No force can shake this great nation, says Xi ...",https://www.hindustantimes.com/world-news/no-f...,China on Tuesday brought out its big guns and ...
2,Saudi Arabia / Turkey,Sometimes I have a coffee and a cigar at hand....,6,"[{'word': 'Khashoggi', 'tag': 'B-PER', 'confid...",World,Saudi operatives suspected of killing the jour...,2019-10-01,2020-08-16-32 12:08:36,Hindustan Times,"Saudi Crown Prince Mohammed bin Salman, a sel...","On secret tape, Saudi journalist Jamal Khashog...",https://www.hindustantimes.com/world-news/on-s...,Saudi operatives suspected of killing the jour...
3,India / Turkey,Erdogan also commissioned a new naval warship ...,10,"[{'word': 'Erdogan', 'tag': 'B-ORG', 'confiden...",World,Turkey has begun work on building a naval wars...,2019-10-01,2020-08-16-32 12:08:42,Hindustan Times,Turkish president Recep Tayyip Erdogan was att...,"Turkey building naval warship for Pakistan, sa...",https://www.hindustantimes.com/world-news/turk...,Turkey has begun work on building a naval wars...
4,China / USA,US government officials say they have charged ...,11,"[{'word': 'US', 'tag': 'B-LOC', 'confidence': ...",World,US government officials say they have charged ...,2019-10-01,2020-08-16-32 12:08:44,Hindustan Times,US Attorney David L Anderson says the charges ...,US citizen accused of spying on behalf of Chin...,https://www.hindustantimes.com/world-news/us-c...,US government officials say they have charged ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,India / China,Whether Xi’s gift to Modi and Bhandari will be...,8707,"[{'word': 'Xi', 'tag': 'B-PER', 'confidence': ...",World,President Xi Jinping will be heading to Chenna...,2020-07-09,2020-08-16-32 12:03:56,Hindustan Times,The Chinese foreign ministry on Thursday said ...,Prez Xi Jinping’s gift for PM Narendra Modi ho...,https://www.hindustantimes.com/world-news/prez...,President Xi Jinping will be heading to Chenna...
1960,Mexico / USA,We didn’t fight. Trump signed and gave López O...,8710,"[{'word': 'Trump', 'tag': 'B-PER', 'confidence...",World,"President Donald Trump, who has denigrated Mex...",2020-07-09,2020-08-16-32 08:41:19,Hindustan Times,Trump’s warm words were in stark contrast to t...,"Trump forgoes insults of past, calls Mexico ch...",https://www.hindustantimes.com/world-news/trum...,"President Donald Trump, who has denigrated Mex..."
1961,China / USA,There’s no president that’s ever stood up to C...,8713,"[{'word': 'China', 'tag': 'B-LOC', 'confidence...",World,The United States is preparing to take additio...,2020-07-09,2020-08-16-32 08:41:15,Hindustan Times,Relations between Washington and Beijing have ...,United States considering additional actions a...,https://www.hindustantimes.com/world-news/unit...,The United States is preparing to take additio...
1962,China / USA,We’re introducing the ‘Preventing China from E...,8714,"[{'word': '‘', 'tag': 'B-MISC', 'confidence': ...",World,A group of 14 US lawmakers have introduced a b...,2020-07-09,2020-08-16-32 08:41:15,Hindustan Times,The bill requires the DNI -- in coordination w...,US lawmakers seek probe into efforts made by C...,http://www.hindustantimes.com/world-

### jackarta

In [33]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/ini.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn = chn.dropna(axis=0).reset_index(drop=True)
chn['doc_id'] = range(0,len(chn))

In [34]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('ini_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\SNUda\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:937: UserWarning: Ignoring URL 'http://www.chictr.org.cn/showproj.aspx?proj=55524Separately,%20Ad5-nCoV,%20a%20vaccine%20candidate%20co-developed%20by%20AMS%20and%20CanSino%20Biologics%20using%20a%20different%20technique,%20is%20under%20phase-2%20clinical%20trial%20in%20China%20and%20has%20won%20approval%20to%20begin%20human%20testing%20in%20Canada.%20A%20Chinese%20military%20research%20institute%20has%20been%20approved%20to%20test%20its%20second%20experimental%20coronavirus%20vaccine%20in%20humans,%20the%20eighth%20candidate%20in%20clinical%20trials%20for%20China%20as%20it%20emerges%20as%20a%20fro

### japantimes

In [35]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/jpn.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
## remove nan category. since category selection
chn['doc_id'] = range(0,len(chn))

In [36]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('jpn_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [37]:
chn

,list_of_countries,input_text,doc_id,sententce_ner,url,title,date,content,category,source,scraping_date,subtitle,lowercase_content
0,Russia / USA,Stone’s trial arose from former special counse...,14,"[{'word': 'Stone', 'tag': 'B-PER', 'confidence...",https://www.japantimes.co.jp/news/2020/02/17/w...,"Over 1,000 ex-Justice Department officials cal...",2020-02-17,"WASHINGTON – More than 1,000 former U.S. Justi...",World,Japantimes,2020-08-20 02:11:52,NaN,"WASHINGTON – More than 1,000 former U.S. Justi..."
1,Russia / UK,LONDON – The U.K. said it will spend £1.2 bill...,16,"[{'word': 'LONDON', 'tag': 'B-LOC', 'confidenc...",https://www.japantimes.co.jp/news/2020/02/17/w...,U.K. to spend $1.6 billion on world's best cli...,2020-02-17,LONDON – The U.K. said it will spend £1.2 bill...,World,Japantimes,2020-08-20 02:11:24,NaN,LONDON – The U.K. said it will spend £1.2 bill...
2,N.Korea / USA,"… That will be discussed with the ICC,” Inform...",19,"[{'word': 'ICC', 'tag': 'B-ORG', 'confidence':...",https://www.japantimes.co.jp/news/2020/02/18/w...,ICC trial in The Hague one option for Sudan's ...,2020-02-18,KHARTOUM – Sudan could send former leader Omar...,World,Japantimes,2020-08-20 02:11:22,NaN,KHARTOUM – Sudan could send former leader Omar...
3,India / Turkey,ISLAMABAD – A high-profile local Taliban figur...,20,"[{'word': 'ISLAMABAD', 'tag': 'B-ORG', 'confid...",https://www.japantimes.co.jp/news/2020/02/18/w...,Pakistan confirms escape of Taliban leader who...,2020-02-18,ISLAMABAD – A high-profile local Taliban figur...,World,Japantimes,2020-08-20 02:11:28,NaN,ISLAMABAD – A high-profile local Taliban figur...
4,Russia / Turkey,TRIPOL – A port in the capital of war-torn Lib...,26,"[{'word': 'Libya', 'tag': 'B-LOC', 'confidence...",https://www.japantimes.co.jp/news/2020/02/19/w...,Port hit by barrage of rocket fire in Libyan c...,2020-02-19,TRIPOL – A port in the capital of war-torn Lib...,World,Japantimes,2020-08-20 02:11:06,NaN,TRIPOL – A port in the capital of war-torn Lib...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,China / USA,Chinese Foreign Ministry spokesperson Zhao Lij...,3109,"[{'word': 'Chinese', 'tag': 'B-MISC', 'confide...",https://www.japantimes.co.jp/news/2020/07/09/w...,Criticism of U.S. pullout from WHO comes from ...,2020-07-09,GENEVA – Top U.S. allies on Wednesday denounc...,World,Japan Times,2020-07-14-28 12:57:21,NaN,GENEVA – Top U.S. allies on Wednesday denounc...
604,Japan / USA,Is it safe to visit the dentist during the COV...,3112,"[{'word': 'COVID-19', 'tag': 'B-MISC', 'confid...",https://www.japantimes.co.jp/news/2020/07/09/w...,Is it safe to visit the dentist during the pan...,2020-07-09,Is it safe to visit the dentist during the COV...,World,Japan Times,2020-07-14-28 12:57:17,NaN,Is it safe to visit the dentist during the COV...
605,Russia / USA,He had recently sought to postpone his surrend...,3116,"[{'word': 'Trump', 'tag': 'B-PER', 'confidence...",https://www.japantimes.co.jp/news/2020/07/11/w...,Trump commutes longtime friend Roger Stone's s...,2020-07-11,Washington – U.S. President Donald Trump calle...,World,Japan Times,2020-07-14-28 12:57:13,NaN,Washington – U.S. President Donald Trump calle...
606,Russia / USA,"Pat Toomey, a Pennsylvania Republican, signale...",3119,"[{'word': 'Pat', 'tag': 'B-PER', 'confidence':...",https://www.japantimes.co.jp/news/2020/07/12/w...,Trump's defiant help for Roger Stone adds to t...,2020-07-12,WASHINGTON – U.S. President Donald Trump’s int...,World,Japan Times,2020-07-14-28 12:57:12,NaN,WASHINGTON – U.S. President Donald Trump’s int...


### koreantimes

In [38]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/kor.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [39]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('kor_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### mexico

In [40]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/max.xlsx')


chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [41]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('max_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [42]:
chn 

,list_of_countries,input_text,doc_id,sententce_ner,content,date,scrap_time,source,title,url,category,subtitle,lowercase_content
0,Mexico / USA,The Mexican peso could fall almost 9% to 21.30...,1,"[{'word': 'Mexican', 'tag': 'B-MISC', 'confide...",The Mexican peso could fall almost 9% to 21.30...,2019-10-10,2020-08-16 17:40:26.794614,vallart_adaily(Mexico),Mexican peso could drop almost 9% to 21.30 per...,https://www.vallartadaily.com/mexican-peso-cou...,world,NaN,The Mexican peso could fall almost 9% to 21.30...
1,Mexico / USA,The migrants are originally from Central Ameri...,2,"[{'word': 'Central', 'tag': 'B-LOC', 'confiden...","Around 2,000 migrants set out in a caravan fro...",2019-10-12,2020-08-16 17:40:25.485192,vallart_adaily(Mexico),"Caravan of 2,000 migrants head toward US from ...",https://www.vallartadaily.com/caravan-of-2000-...,world,NaN,"Around 2,000 migrants set out in a caravan fro..."
2,Mexico / USA,Mexico’s military police on Saturday halted an...,3,"[{'word': 'Mexico', 'tag': 'B-LOC', 'confidenc...",Mexico’s military police on Saturday halted an...,2019-10-13,2020-08-16 17:40:23.711471,vallart_adaily(Mexico),Mexico military police halt migrant caravan on...,https://www.vallartadaily.com/mexico-military-...,world,NaN,Mexico’s military police on Saturday halted an...
3,Mexico / USA,“Not only did they get the government to relea...,7,"[{'word': 'Ovidio', 'tag': 'B-PER', 'confidenc...",Mexican officials on Friday admitted they had ...,2019-10-19,2020-08-16 17:40:17.573795,vallart_adaily(Mexico),Mexican Cartel outsmarts and outpowers newly f...,https://www.vallartadaily.com/mexican-cartel-o...,world,NaN,Mexican officials on Friday admitted they had ...
4,Mexico / USA,This is due to the entry of the Cocos plate (o...,12,"[{'word': 'Cocos', 'tag': 'B-LOC', 'confidence...",In the last hours a series of earthquakes have...,2019-10-23,2020-08-16 17:40:12.922405,vallart_adaily(Mexico),"Guerrero, Mexico records a “seismic swarm”",https://www.vallartadaily.com/guerrero-mexico-...,world,NaN,In the last hours a series of earthquakes have...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Mexico / USA,"In Mexico, their favorite crops are corn, suga...",384,"[{'word': 'Mexico', 'tag': 'B-LOC', 'confidenc...",As though the coronavirus pandemic hasn’t been...,2020-07-19,2020-08-16 18:28:21.052898,vallart_adaily(Mexico),Mexico prepares for the arrival of a swarm of ...,https://www.vallartadaily.com/mexico-prepares-...,world,NaN,As though the coronavirus pandemic hasn’t been...
95,Mexico / USA,” As part of the entry into force of the trade...,385,"[{'word': 'Mexico', 'tag': 'B-LOC', 'confidenc...",As part of the entry into force of the trade a...,2020-07-02,2020-08-16 18:28:40.595193,vallart_adaily(Mexico),Canadian business becomes first to invest in M...,https://www.vallartadaily.com/canadian-busines...,world,NaN,As part of the entry into force of the trade a...
96,Mexico / USA,He has generally downplayed the pandemic’s thr...,389,"[{'word': 'Mexico', 'tag': 'B-LOC', 'confidenc...",Mexican President Andrés Manuel López Obrador ...,2020-07-23,2020-08-16 18:28:19.952137,vallart_adaily(Mexico),Mexico’s president dismissive of wearing mask ...,https://www.vallartadaily.com/mexicos-presiden...,world,NaN,Mexican President Andrés Manuel López Obrador ...
97,Mexico / USA,"Mexico’s coronavirus outbreak rose by about 6,...",392,"[{'word': 'Mexico', 'tag': 'B-LOC', 'confidenc...","Mexico’s coronavirus outbreak rose by about 6,...",2020-07-03,2020-08-16 18:28:38.128700,vallart_adaily(Mexico),Mexico reverses course on some reopening as CO...,https://www.vallartadaily.com/mexico-reverses-...,world,NaN,"Mexico’s coronavirus outbreak rose by about 6,..."


## taipei

In [43]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/tai.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [44]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('tai_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [45]:
chn

,list_of_countries,input_text,doc_id,sententce_ner,category,content,date,scrap_time,source,subtitle,title,url,lowercase_content
0,Japan / S.Korea,The Japanese embassy in Seoul last week began ...,5,"[{'word': 'Japanese', 'tag': 'B-MISC', 'confid...",World News,Japan’s embassy in South Korea has begun posti...,2019-10-01,2020-08-16-32 16:18:12,Taipeitimes(Taiwan),TIGHTENING CONTROLS:South Koreans have become ...,Japan lists Fukushima radiation levels,https://www.taipeitimes.com/News/taiwan/archiv...,Japan’s embassy in South Korea has begun posti...
1,China / USA,China is planning tighter visa restrictions fo...,17,"[{'word': 'China', 'tag': 'B-LOC', 'confidence...",World News,China is planning tighter visa restrictions fo...,2019-10-10,2020-08-16-32 16:08:40,Taipeitimes(Taiwan),NaN,China to restrict visas due to US curbs: sources,https://www.taipeitimes.com/News/taiwan/archiv...,China is planning tighter visa restrictions fo...
2,Turkey / USA,Turkey has long threatened an attack on the Ku...,19,"[{'word': 'Turkey', 'tag': 'B-LOC', 'confidenc...",World News,Syrian Kurds yesterday issued a “general mobil...,2019-10-10,2020-08-16-32 16:08:47,Taipeitimes(Taiwan),BREAKING POINT?A senior Turkish official said ...,Kurds mobilize as Turkey poised to attack,https://www.taipeitimes.com/News/taiwan/archiv...,Syrian Kurds yesterday issued a “general mobil...
3,Japan / USA,“We have gained access to a technical revoluti...,21,"[{'word': 'Sara', 'tag': 'B-PER', 'confidence'...",World News,Three scientists yesterday won the Nobel Prize...,2019-10-10,2020-08-16-32 16:08:51,Taipeitimes(Taiwan),NaN,Three win Nobel for work on lithium-ion batteries,https://www.taipeitimes.com/News/taiwan/archiv...,Three scientists yesterday won the Nobel Prize...
4,China / USA,"“Once again, Vietnam demands that China immedi...",25,"[{'word': 'Vietnam', 'tag': 'B-LOC', 'confiden...",World News,Vietnam has accused a Chinese oil-surveying ve...,2019-10-11,2020-08-16-32 16:07:51,Taipeitimes(Taiwan),NaN,Chinese escalating tensions in South China Sea...,https://www.taipeitimes.com/News/taiwan/archiv...,Vietnam has accused a Chinese oil-surveying ve...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Australia / China,An Amnesty International report last year said...,4599,"[{'word': 'Amnesty', 'tag': 'B-ORG', 'confiden...",World News,Australia yesterday warned its citizens that t...,2020-07-08,2020-08-16-32 10:46:30,Taipeitimes(Taiwan),NaN,Australia warns of ‘arbitrary detention’ risk ...,https://www.taipeitimes.com/News/taiwan/archiv...,Australia yesterday warned its citizens that t...
620,Italy / China,"The filing, submitted on behalf of the “East T...",4601,"[{'word': 'East', 'tag': 'B-LOC', 'confidence'...",World News,A group of Uighurs in exile has submitted evid...,2020-07-08,2020-08-16-32 10:46:42,Taipeitimes(Taiwan),‘GENOCIDE’:As some of the alleged crimes took ...,Exiled Uighurs call on ICC to probe Xinjiang,https://www.taipeitimes.com/News/taiwan/archiv...,A group of Uighurs in exile has submitted evid...
621,Italy / USA,"Keen to improve Italy’s digitalization, the de...",4602,"[{'word': 'Italy', 'tag': 'B-LOC', 'confidence...",World News,The Italian government yesterday approved a pa...,2020-07-08,2020-08-16-32 10:46:34,Taipeitimes(Taiwan),‘SIMPLIFICATION DECREE’:Touted as the ‘mother ...,Italian Cabinet passes measures to slash red tape,https://www.taipeitimes.com/News/taiwan/archiv...,The Italian government yesterday approved a pa...
622,China / USA,“China describes Fox Hunt as some kind of inte...,4615,"[{'word': 'China', 'tag': 'B-LOC', 'confidence...",World News,Chinese agents have been pursuing hundreds of ...,2020-07-09,2020-08-16-32 10:45:32,Taipeitimes(Taiwan),"FOX HUNT:To suppress dissent, Chinese living a...",China forcing dissenters to return home: FBI head,https://www.taipeitimes.com/News/taiwan/archiv...,Chinese agents have been pursuing hundreds of ...


## turkey

In [67]:
chn = pandas.read_pickle('C:/Users/SNUda/Downloads/juwon/raw_data/tuk.p')
chn['source'] = "dailysabah(turkey)"
chn['subtitle'] = ""
chn.columns = ['title', 'date', 'category', 'content', 'url', 'source', 'subtitle']
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [68]:
chn

,title,date,category,content,url,source,subtitle,doc_id
0,US prosecutors expand charges against 2 former...,2020-07-30,"ISTANBUL, Americas",U.S. prosecutors have expanded charges against...,https://www.dailysabah.com/world/americas/us-p...,dailysabah(turkey),,0
1,Careless young people driving some COVID-19 sp...,2020-07-30,"World, COVID-19, PANDEMIC, world, health, orga...",The World Health Organization (WHO) on Thursda...,https://www.dailysabah.com/world/careless-youn...,dailysabah(turkey),,1
2,EU to remove Algeria from COVID-19 safe countr...,2020-07-30,"Europe, European, Union, COVID-19, PANDEMIC, A...",The European Union countries agreed Thursday t...,https://www.dailysabah.com/world/europe/eu-to-...,dailysabah(turkey),,2
3,German elite commando forces dismantled due to...,2020-07-30,"BERLIN, Europe, right-wing, extremism, German,...",A company belonging to the German army's elite...,https://www.dailysabah.com/world/europe/german...,dailysabah(turkey),,3
4,Trump suggests delaying US presidential elections,2020-07-30,"WASHINGTON, Americas",U.S. President Donald Trump is for the first t...,https://www.dailysabah.com/world/americas/trum...,dailysabah(turkey),,4
...,...,...,...,...,...,...,...,...
6603,Israel intensifies violence against Palestinia...,2019-07-01,"ISTANBUL, World, HomePage",Israeli police clashed Sunday with residents o...,https://www.dailysabah.com/mideast/2019/07/01/...,dailysabah(turkey),,6603
6604,Yemenis rally against UAE-backed forces,2019-07-01,"ISTANBUL, World, World",Hundreds of protesters on Sunday rallied on Ye...,https://www.dailysabah.com/mideast/2019/07/01/...,dailysabah(turkey),,6604
6605,Italy at odds with Germany over migrant aid ship,2019-07-01,"ISTANBUL, Europe, World",A German charity rescue ship was impounded and...,https://www.dailysabah.com/europe/2019/07/01/i...,dailysabah(turkey),,6605
6606,Venezuelan rival sides set to break deadlock,2019-07-01,"BOGOTA, Americas, World",Negotiations aimed at breaking Venezuela's pol...,https://www.dailysabah.com/americas/2019/07/01...,dailysabah(turkey),,6606


In [69]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge([ner_df, chn])
chn.to_excel('turk_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


TypeError: merge() missing 1 required positional argument: 'right'

### Sputnik

In [46]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/rus.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [47]:
chn

,date,title,subtitle,content,category,source,url,scrap_time,doc_id
0,2019-07-01,50 IAF AN-32 Military Transport Aircraft Await...,NaN,The Indian Air Force has said it will replace ...,Military & Intelligence,Sputnik,https://sputniknews.com/military/2019070110761...,2020-07-19-28 10:43:24,0
1,2019-07-01,Abrams Tanks Arrive in US Capital for July 4 C...,NaN,At least two M1A1 Abrams tanks from the US Arm...,US,Sputnik,https://sputniknews.com/us/201907011076119214-...,2020-07-19-28 10:43:21,1
2,2019-07-01,"Afghan Peace: Germany, Qatar to Host Taliban-U...",NaN,WASHINGTON (Sputnik) - Germany and Qatar have ...,World,Sputnik,https://sputniknews.com/world/2019070110761183...,2020-07-19-28 10:43:32,2
3,2019-07-01,At Least 9 Injured as Yemen's Houthis Target S...,NaN,Yemen's Houthi faction said early on Tuesday t...,Middle East,Sputnik,https://sputniknews.com/middleeast/20190701107...,2020-07-19-28 10:43:28,3
4,2019-07-01,‘Big Waves’: China Confirms Successful Testing...,NaN,"In early June, residents in several provinces ...",Military & Intelligence,Sputnik,https://sputniknews.com/military/2019070110761...,2020-07-19-28 10:49:17,4
...,...,...,...,...,...,...,...,...,...
9777,2020-07-19,Two African-American Women Arrested for Smeari...,NaN,The giant inscription painted by activists wit...,US,Sputnik,https://sputniknews.com/us/202007191079928073-...,2020-07-19-28 10:40:29,9777
9778,2020-07-19,Two Rockets Fall Near US Embassy Inside Baghda...,NaN,BAGHDAD (Sputnik) - Two rockets fell on Sunday...,Middle East,Sputnik,https://sputniknews.com/middleeast/20200719107...,2020-07-19-28 15:53:58,9778
9779,2020-07-19,UK Hospital in Brighton on Lockdown After Man ...,NaN,"According to preliminary reports, the suspect,...",UK,Sputnik,https://sputniknews.com/uk/202007191079928212-...,2020-07-19-28 10:40:31,9779
9780,2020-07-19,UK PM Johnson Compares Possible Second COVID-1...,NaN,"British media reported earlier this week, citi...",UK,Sputnik,https://sputniknews.com/uk/202007191079928300-...,2020-07-19-28 15:53:59,9780


In [48]:
Counter(chn['category']).most_common(30)

[('US', 2391),
 ('Military & Intelligence', 2017),
 ('Middle East', 1058),
 ('World', 910),
 ('Europe', 870),
 ('Asia & Pacific', 777),
 ('Latin America', 286),
 ('UK', 225),
 ('Russia', 219),
 ('Viral', 181),
 ('Opinion', 148),
 ('Tech', 138),
 ('Africa', 135),
 ('India', 102),
 ('Business', 88),
 ('Society', 83),
 ('Cartoons', 48),
 ('Sport', 39),
 ('Environment', 31),
 ('Videoclub', 21),
 ('Life', 7),
 ('News', 7),
 ('Interviews', 1)]

In [49]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        ner_df['sententce_ner'][a] = [each for each in model.predict(ner_df['input_text'][a]) if each['tag']!='O']
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('rus_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### singapore

In [50]:
chn = pd.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/sgn.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [51]:
chn

,date,title,time,subtitle,content,category,source,url,scrap_time,doc_id
0,2020-03-29,Ireland's calm before the surge,2020-03-29,NaN,"About this time four weeks ago, Ireland's care...",CORONAVIRUS,Straitstimes,https://www.straitstimes.com/opinion/irelands-...,2020-07-19-28 13:59:08,0
1,2020-04-08,Public health is a national security issue,2020-04-08,NaN,"In India, the world's most elaborate lockdown ...",COVID-19,Straitstimes,https://www.straitstimes.com/opinion/st-editor...,2020-07-19-28 13:58:20,1
2,2020-04-10,"To stay safe and well, just stay at home!",2020-04-10,NaN,It was just two days into the Great Singapore ...,COVID-19,Straitstimes,https://www.straitstimes.com/asia/to-stay-safe...,2020-07-19-28 13:58:15,2
3,2020-04-12,Housebound in Singapore: Reflecting on circuit...,2020-04-12,NaN,As circuit breaker measures continue to tighte...,CORONAVIRUS,Straitstimes,https://www.straitstimes.com/opinion/houseboun...,2020-07-19-28 13:58:01,3
4,2020-04-14,Care for foreign workers is essential,2020-04-14,NaN,"The recent sharp rise in coronavirus cases, wi...",ST EDITORIAL,Straitstimes,https://www.straitstimes.com/opinion/st-editor...,2020-07-19-28 13:57:57,4
...,...,...,...,...,...,...,...,...,...,...
5413,2020-07-20,Work in the age of the coronavirus,2020-07-20,NaN,Eve (not her real name) is a rather popular an...,COVID-19,Straitstimes,https://www.straitstimes.com/opinion/work-in-t...,2020-07-19-28 13:43:07,5413
5414,2020-07-20,World in transition - the 4 big challenges,2020-07-20,NaN,"Rapid change is now a constant, globally impor...",COVID-19,Straitstimes,https://www.straitstimes.com/opinion/world-in-...,2020-07-19-28 13:41:44,5414
5415,2020-07-20,World must unite to flatten the curve,2020-07-20,NaN,"Four months into the year, the world's worst p...",US-CHINA TIES,Straitstimes,https://www.straitstimes.com/opinion/st-editor...,2020-07-19-28 13:58:29,5415
5416,2020-07-20,Youth are the pioneers of post-Covid-19 Singapore,2020-07-20,NaN,Nearly a century since the phrase first emerge...,COVID-19,Straitstimes,https://www.straitstimes.com/opinion/youth-are...,2020-07-19-28 13:41:44,5416


In [52]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('sgn_country.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### saudi arabia

In [39]:
chn = pandas.read_pickle('C:/Users/SNUda/Downloads/juwon/raw_data/sau.p')


for label in range(len(chn['date'])):
     chn['date'][label]= chn['date'][label].strftime("%Y-%m-%d")
chn['source'] = "arabnews(Saudi)"
chn['subtitle'] = ""
chn['category'] = "world"
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

chn = chn.loc[chn["date"] >= "2019-07-01"].reset_index(drop=True)
chn = chn.loc[chn["date"] <= "2020-07-30"].reset_index(drop=True)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\SNUda\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('sau_ner.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### France

In [55]:
chn = pandas.read_pickle('C:/Users/SNUda/Downloads/juwon/raw_data/frn.p')
for label in range(len(chn['date'])):
     chn['date'][label]= chn['date'][label].strftime("%Y-%m-%d")
chn['source'] = "france24(France)"
chn['subtitle'] = ""
#chn.columns = ['title', 'date', 'category', 'content', 'url', 'source', 'subtitle']
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn = chn[~chn['category'].str.contains("Sport|Business|Culture|Media")].reset_index(drop=True)
Counter(chn['category']).most_common(30)
chn['doc_id'] = range(0,len(chn))

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
chn

,title,date,category,content,url,source,subtitle,doc_id
0,Trump's fellow Republicans reject his idea of ...,2020-07-30,Americas,Critics and even Trump's allies dismissed the ...,https://www.france24.com/en/20200730-usa-donal...,france24(France),,0
1,Law enforcement behind half of all violence ag...,2020-07-30,Europe,According to a report by UNHCR and the Danish ...,https://www.france24.com/en/20200730-law-enfor...,france24(France),,1
2,US economy shrinks record 33% in second quarte...,2020-07-30,Americas,The Commerce Department’s estimate of the seco...,https://www.france24.com/en/20200730-us-econom...,france24(France),,2
3,NASA launches Mars rover Perseverance to seek ...,2020-07-30,Americas,A United Launch Alliance Atlas V rocket took o...,https://www.france24.com/en/20200730-nasa-laun...,france24(France),,3
4,Coronavirus cluster in western France pits you...,2020-07-30,France,An outbreak among 18- to 25-year-olds at a sea...,https://www.france24.com/en/20200730-coronavir...,france24(France),,4
...,...,...,...,...,...,...,...,...
3129,Dozens wounded as car bomb rips through diplom...,2019-07-01,Asia / Pacific,The rush-hour explosion sent a plume of smoke ...,https://www.france24.com/en/20190701-afghanist...,france24(France),,3129
3130,EU leaders meeting in Brussels divided over wh...,2019-07-01,Europe,France's President Emmanuel Macron and Germany...,https://www.france24.com/en/20190701-eu-leader...,france24(France),,3130
3131,Israeli air strikes in Syria kill several civi...,2019-07-01,Middle East,The raids near Damascus and in Homs province l...,https://www.france24.com/en/20190701-israeli-m...,france24(France),,3131
3132,"Protesters, police clash in Hong Kong before h...",2019-07-01,Asia / Pacific,The international financial hub has been shake...,https://www.france24.com/en/20190701-protester...,france24(France),,3132


In [57]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('frn_ner.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### south africa

In [58]:
chn = pandas.read_excel('C:/Users/SNUda/Downloads/juwon/raw_data/zaf.xlsx')

chn['source'] = "news24(South Africa)"
chn['subtitle'] = ""
chn['category'] = "world"
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn['doc_id'] = range(0,len(chn))

In [59]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('zaf_ner.xlsx', index = False)

C:\Users\SNUda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## 호주

In [22]:
chn = pandas.read_excel('C:/Users/SNUda/Downloads/aut.xlsx')
chn['title'] = chn['title'].str.replace('\n', ' ')
chn['content'] = chn['content'].str.replace('\n', ' ')
chn = chn.drop_duplicates(subset=['title'])
chn = chn.dropna(subset=['content']).reset_index(drop=True) 
chn = chn.dropna(subset=['title']).reset_index(drop=True)  
chn['doc_id'] = range(0,len(chn))

In [ ]:
ner_df = quasiNER_extractor3(chn, 'content')
ner_df['sententce_ner'] = ''
for a in range(len(ner_df)):
    try :
        temp = ner_df['input_text'][a].split('.')
        x =[]
        for b in temp:
            [x.append(each) for each in model.predict(b) if each['tag']!='O']
            ner_df['sententce_ner'][a] = x
    except :
        ner_df['sententce_ner'][a] = ""
chn = pd.merge(ner_df, chn)
chn.to_excel('aus_ner.xlsx', index = False)